In [ ]:
import pandas as pd
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from urllib import request
import json
import requests

Initiate browser, query word, get network flow with specific name, get its GET Request URL, download the file with the words name in it.

In [ ]:
# writes all the logs into a 'network_log.json' file.
def writeLogs(logs):
    # Opens a writable JSON file and writes the logs in it
    with open("network_log.json", "w", encoding="utf-8") as f:
        f.write("[")

        # Iterates every logs and parses it using JSON
        for log in logs:
            network_log = json.loads(log["message"])["message"]

            # Checks if the current 'method' key has any
            # Network related value.
            if("Network.response" in network_log["method"]
                    or "Network.request" in network_log["method"]
                    or "Network.webSocket" in network_log["method"]):

                # Writes the network log to a JSON file by
                # converting the dictionary to a JSON string
                # using json.dumps().
                f.write(json.dumps(network_log)+",")
        f.write("{}]")

        return None


# searches the logs for the specific link
def getVoiceLink(logs):
    # Iterate the logs
    for log in logs:

        # Except block will be accessed if any of the
        # following keys are missing.
        try:
            # URL is present inside the following keys
            url = log["params"]["request"]["url"]

            if 'voice' in url:
                return url

        except Exception as e:
            pass

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--mute-audio')

In [ ]:
capabilities = DesiredCapabilities.CHROME
capabilities["goog:loggingPrefs"] = {"performance": "ALL"}

In [ ]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), desired_capabilities=capabilities, options=options)

In [ ]:
word = 'нормально'
mainURL = 'https://context.reverso.net/translation/russian-english/' + word
driver.get(mainURL)

In [ ]:
driver.find_element(By.XPATH, '//*[@id="search-links"]/button[1]').click()

In [ ]:
# Gets all the logs from performance in Chrome
logs = driver.get_log("performance")

writeLogs(logs)

In [ ]:
print("Getting User-Agent")
userAgent = driver.execute_script("return navigator.userAgent")

In [ ]:
print("Quitting Selenium WebDriver")
driver.quit()

In [ ]:
# Read the JSON File and parse it using
# json.loads() to find the urls containing images.
json_file_path = "network_log.json"
with open(json_file_path, "r", encoding="utf-8") as f:
    logs = json.loads(f.read())

In [ ]:
link = getVoiceLink(logs)

headers = {'User-Agent':userAgent}

response = requests.get(link, headers=headers)

In [ ]:
with open(word + '.mp3', 'wb') as f:
    f.write(response.content)